In [142]:
import pandas as pd

In [143]:
train_df=pd.read_csv('Train_Data.csv')
test_df=pd.read_csv('Test_Data.csv')

In [144]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.impute import KNNImputer

In [145]:
train_df.drop('SEQN',axis=1,inplace=True)
train_df['Is_Senior']=[0 if x =='Adult' else 1 for x in train_df['age_group']]
train_df.drop('age_group',axis=1,inplace=True)

target_col=['Is_Senior']
feature_cols=[col for col in train_df.columns if col not in target_col]


In [146]:
it_imputer= KNNImputer()
train_df[feature_cols]=it_imputer.fit_transform(train_df[feature_cols])

In [147]:
import numpy as np

def detect_outliers(data):
    
    q1,q3=np.percentile(data,[25,75])
    
    iqr=q3-q1
    lower_bound=q1-(1.5*iqr)
    upper_bound=q3+(1.5*iqr)
    
    data = np.array(data, dtype=float)

    data[(data < lower_bound) | (data > upper_bound)] = np.nan

    data=pd.Series(data)
    
    return data
    

In [148]:
train_df['BMXBMI']= detect_outliers(train_df['BMXBMI'])

In [149]:
train_df['LBXGLU'] = detect_outliers(train_df['LBXGLU'])
train_df['LBXGLT'] = detect_outliers(train_df['LBXGLT'])
train_df['LBXIN'] = detect_outliers(train_df['LBXIN'])

In [150]:
train_df[feature_cols]=it_imputer.fit_transform(train_df[feature_cols])

In [151]:
train_df.isnull().sum()

RIAGENDR     0
PAQ605       0
BMXBMI       0
LBXGLU       0
DIQ010       0
LBXGLT       0
LBXIN        0
Is_Senior    0
dtype: int64

In [152]:
train_df.loc[train_df['PAQ605'] > 2, 'PAQ605'] = 2


In [153]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_df[feature_cols],train_df[target_col],test_size=0.2)

In [154]:
from sklearn.ensemble import AdaBoostClassifier
# abc = AdaBoostClassifier(n_estimators=40,learning_rate=0.1,random_state=0,class_weight='balanced')

from sklearn.tree import DecisionTreeClassifier
base_est = DecisionTreeClassifier(max_depth=3, class_weight='balanced', random_state=0)
abc = AdaBoostClassifier(estimator=base_est, n_estimators=40, learning_rate=0.1, random_state=0)

In [155]:
model1=abc.fit(X_train,y_train)

c:\Users\ujjwal krishna\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ujjwal krishna\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [156]:
y_pred = model1.predict(X_test)

In [157]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6776649746192893

In [158]:
test_df.drop('SEQN',axis=1,inplace=True)

test_df[feature_cols]=it_imputer.fit_transform(test_df[feature_cols])

test_df['BMXBMI']= detect_outliers(test_df['BMXBMI'])
test_df['LBXGLU'] = detect_outliers(test_df['LBXGLU'])
test_df['LBXGLT'] = detect_outliers(test_df['LBXGLT'])
test_df['LBXIN'] = detect_outliers(test_df['LBXIN'])

test_df[feature_cols]=it_imputer.fit_transform(test_df[feature_cols])

test_df.loc[test_df['PAQ605'] > 2, 'PAQ605'] = 2

In [159]:
y_pred = model1.predict(test_df)

In [160]:
result = pd.DataFrame({
    'age_group': y_pred
})

result.to_csv('result6.csv',index=False)

In [170]:
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier(
    n_estimators=250,
    min_samples_split=5,     # Reduced from 20
    min_samples_leaf=3,      # Reduced from 10
    max_depth=10,            # Add depth limit
    class_weight='balanced',
    random_state=42
    )

model2.fit(X_train,y_train)

c:\Users\ujjwal krishna\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=3, min_samples_split=5,
                       n_estimators=250, random_state=42)

In [171]:
model2.score(X_train,y_train)

0.8912213740458015

In [172]:
y_pred = model2.predict(test_df)

In [173]:
result = pd.DataFrame({
    'age_group': y_pred
})

result.to_csv('result4.csv',index=False)